In [1]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.1 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=8e2f9e2ee0bc1e43667b3cb085a98dd1bd2d7cad5358f29a83be08724c4ed5e9
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=a77a836dfa27fd3326198bab14a1803856592df051b55c

In [70]:
import pandas as pd
import numpy as np
import random
import re
import requests
from keybert import KeyBERT
from scipy import spatial 
from sentence_transformers import SentenceTransformer, models

# Make Sample list(Ko, En)

def Make_En():
    Table = pd.read_csv('/content/drive/MyDrive/task1/task1/Sample.csv')
    Table = Table.iloc[:, 1:]
    Table.dropna(axis = 0, inplace = True)
    Table_list = list(Table.comment)

    Table_list_en = []
    for elem_ko in Table_list:
        elem_en = get_translate(elem_ko)
        Table_list_en.append(elem_en)

    return Table, Table_list, Table_list_en

# Translate Ko -> En

def get_translate(text):
    client_id = '9aorPMFuECuHYdvjMjYV'
    client_secret = 'DYCcQFJTt7'
    data = {'text' : text,
            'source' : 'ko',
            'target' : 'en'}
    url = "https://openapi.naver.com/v1/papago/n2mt"
    header = {'X-Naver-Client-Id' : client_id,
              'X-Naver-Client-Secret' : client_secret}
    response = requests.post(url, headers = header, data = data)
    rescode = response.status_code
    if (rescode == 200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText']) # return value
        return trans_data
    else:
        print('Error Code:', rescode)

# Make Keyword

def KeyWordExtracting_Single(doc_en):
     kw_model = KeyBERT()
     keywords_1 = kw_model.extract_keywords(doc_en, keyphrase_ngram_range=(2, 4) , use_mmr=True, diversity=0.60, top_n = 5)
     keywords_2 = kw_model.extract_keywords(doc_en, keyphrase_ngram_range=(2, 4) , use_maxsum=True, nr_candidates=20, top_n = 3)

     return keywords_1, keywords_2

def KeyWordExtracting_Multiple(doc_en):
    kw_model = KeyBERT()
    doc_modified = doc_en.replace(' and', ',').replace(' or', ',')
    doc_list = doc_modified.split(',')
    KEYWORDS = list()
    for index, doc in enumerate(doc_list):
        if len(doc.split()) <= 2:
            continue
        else:
            keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 4) , use_mmr=True, diversity=0.75, top_n = 5)
            keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 4) , use_maxsum=True, nr_candidates=20, top_n = 3)
            KEYWORDS.append((keywords_1, keywords_2))
    return KEYWORDS

def best_cos_sim(table_list: list, keyword_list_1: list, keyword_list_2: list):
    st_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    best_list = []

    # column = 테이블 컬럼명, keyword_list = 추출된 키워드 리스트
    # 추출된 키워드 element 별 해당 컬럼명과 코사인 유사도 비교 & 평균
    # 키워드 리스트 element 들과 컬럼의 유사도 평균값들을 리스트에 저장
    # 저장된 리스트에서 가장 높은 평균 코사인 유사도를 가진 영문 번역 컬럼 인덱스 가져옴
    # return 값으로 제일 높은 평균 코사인 유사도를 가진 영문 번역 컬럼 반환

    for column in table_list:
        column_embedding = st_model.encode(column).flatten()
        sim_total = 0
        for keyword in keyword_list_1:
            keyword = str(keyword)
            keyword = keyword.split(',')[0].strip('(')
            keyword_embedding = st_model.encode(keyword).flatten()
            sim = 1 - spatial.distance.cosine(column_embedding, keyword_embedding) # compute cosine similarity between two vectors
            sim_total += sim
        for keyword in keyword_list_2:
            keyword = str(keyword)
            keyword = keyword.split(',')[0].strip('(')
            keyword_embedding = st_model.encode(keyword).flatten()
            sim = 1 - spatial.distance.cosine(column_embedding, keyword_embedding) # compute cosine similarity between two vectors
            sim_total += sim
        sim_mean = sim_total / len(keyword_list_1 + keyword_list_2)
        best_list.append(sim_mean)
    best_index = best_list.index(max(best_list))
    return table_list[best_index]

def Make_Q_En(Table, doc_en, best_en, Table_list_en, Table_list):
    ko_index = Table_list_en.index(best_en)
    best_ko = Table_list[ko_index]
    print(f'best translation : {best_en} \nKorean: {best_ko}')
    best_row_Table = Table[Table['comment'] == best_ko]
    if len(best_row_Table) != 1:
        Table_name = list(best_row_Table['table_name'])[0]
        col_name = list(best_row_Table['col_name'])[0]
        type_name = list(best_row_Table['data_type'])[0]
        prompt = doc_en +'\n'+f'Table_name :{Table_name}, Column_name : {col_name}, Type : {type_name}'
    else:
        Table_name = list(best_row_Table['table_name'])[0]
        col_name = list(best_row_Table['col_name'])[0]
        type_name = list(best_row_Table['data_type'])[0]
        prompt = doc_en +'\n'+f'Table_name :{Table_name}, Column_name : {col_name}, Type : {type_name}'
    return prompt

def Make_Q_En_Multiple(Table, doc_en, best_en_list: list, Table_list_en, Table_list):
    best_prompt = []
    prompt = doc_en
    for best_en in best_en_list:
        ko_index = Table_list_en.index(best_en)
        best_ko = Table_list[ko_index]
        print(f'best translation : {best_en} \nKorean: {best_ko}')
        best_row_Table = Table[Table['comment'] == best_ko]

        if len(best_row_Table) != 1:
            Table_name = list(best_row_Table['table_name'])[0]
            col_name = list(best_row_Table['col_name'])[0]
            type_name = list(best_row_Table['data_type'])[0]
            best_prompt.append((best_en, Table_name, col_name, type_name))
        else:
            Table_name = list(best_row_Table['table_name'])[0]
            col_name = list(best_row_Table['col_name'])[0]
            type_name = list(best_row_Table['data_type'])[0]
            best_prompt.append((best_en, Table_name, col_name, type_name))
    for bundle in best_prompt:
        best_en, Table_name, col_name, type_name = bundle
        prompt += '\n'+f'Table_name of {best_en}:{Table_name}, Column_name of {best_en} : {col_name}, Type of {col_name} : {type_name}'
    return prompt




def prompt_generation(doc): #prompt generation
    Table, Table_list, Table_list_en = Make_En()
    
    doc_en = get_translate(doc)
    #identify single, multiple
    if ('and' in doc_en) or ('or' in doc_en):
        keywords_list = KeyWordExtracting_Multiple(doc_en)
        print(keywords_list)

        best_en_list = []

        for keywords in keywords_list:
            keywords_1, keywords_2 = keywords
            best_en = best_cos_sim(Table_list_en, keywords_1, keywords_2)
            best_en_list.append(best_en)
        prompt = Make_Q_En_Multiple(Table, doc_en, best_en_list, Table_list_en, Table_list)
        return prompt
                    

    else:
        keywords_1, keywords_2 = KeyWordExtracting_Single(doc_en)
        best_en = best_cos_sim(Table_list_en, keywords_1, keywords_2)
        prompt = Make_Q_En(best_en, Table_list_en, Table_list)
        print(prompt)
        return prompt


In [49]:
def prompt_generation(doc):
    Table, Table_list, Table_list_en = Make_En()
    
    doc_en = get_translate(doc)
    #identify single, multiple
    if ('and' in doc_en) or ('or' in doc_en):
        keywords_list = KeyWordExtracting_Multiple(doc_en)
        print(keywords_list)

        best_en_list = []

        for keywords in keywords_list:
            keywords_1, keywords_2 = keywords
            best_en = best_cos_sim(Table_list_en, keywords_1, keywords_2)
            best_en_list.append(best_en)
        prompt = Make_Q_En_Multiple(Table, doc_en, best_en_list, Table_list_en, Table_list)
        return prompt
                    

    else:
        keywords_1, keywords_2 = KeyWordExtracting_Single(doc_en)
        best_en = best_cos_sim(Table_list_en, keywords_1, keywords_2)
        prompt = Make_Q_En(best_en, Table_list_en, Table_list)
        print(prompt)
        return prompt

In [68]:
doc = input()
doc_en = get_translate(doc)
if ('and' in doc_en) or ('or' in doc_en):
    keywords_list = KeyWordExtracting_Multiple(doc_en)

네이버페이 회원번호가 0으로 시작하고 은행명이 국민은행인 행들을 추출해 줘


[([('naver pay membership number', 0.8702),
   ('pay membership number starts', 0.7353),
   ('membership number', 0.5486),
   ('pay membership', 0.5221),
   ('number starts', 0.433)],
  [('naver pay', 0.6179), ('membership number starts', 0.6299)]),
 ([('bank kookmin bank', 0.9279),
   ('kookmin bank', 0.9076),
   ('bank kookmin', 0.8887)],
  [('kookmin bank', 0.9076), ('bank kookmin bank', 0.9279)])]

In [67]:
doc_en

'Naver Pay membership number starts with 0 and the bank name is Kookmin Bank'

In [56]:
best_list = []
for keywords in keywords_list:
    keywords_1, keywords_2 = keywords
    best_en = best_cos_sim(keywords_)

'Naver Pay membership number starts with 0, the bank name is Kookmin Bank'

In [40]:
B = 'Extract the lines where the membership number of Navi Pay starts with 0 and the payment product name starts with A'
if 'and' in B or 'or' in B:
    print('X')
else:
    print('O')

X
